# Persistence diagram and barcode- plotly

Create a persistence diagram from the persistent homology output.

In [1]:
## Load packages 

import numpy as np
import sys
import pandas as pd
import networkx as nx
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
import aptviz_themes
from supporting import *
import plotly.io as pio

# Load ExHACT visualization themes
pio.templates.default = "aptviz"
my_charcoal = "#3f4142"

In [3]:
## Generate fake data
n_nodes = 50
n_simps = [n_nodes, 100, 150, 100, 20, 5]
max_dim = len(n_simps)-1

fsc_df = create_fake_fsc_df(n_nodes, n_simps, max_dim)
fsc_df.head()

Created df with length 425. Expected 425.


,cell_id,dim,nodes,weight,faces
0,0,0.0,[0],5.709961,[]
1,1,0.0,[1],5.433325,[]
2,2,0.0,[2],5.991614,[]
3,3,0.0,[3],5.136186,[]
4,4,0.0,[4],5.511212,[]


In [ ]:
## FOR DEV ONLY ##
## Generate fake barcode as a pandas df
# Data -- pandas tidy data frame again so that we can add extra hover stuff later
# Note working only with ranks for ease. All should work fine because I have make the ranks floats :)

n_bars = 100
max_filtration = np.max(fsc["rank"])
max_dim = 7

bar_id = np.arange(n_bars)
bar_dim = np.random.randint(0,max_dim,size=n_bars)
bar_birth = np.random.choice(fsc[fsc.dim==1]["rank"],n_bars)
bar_death = np.random.choice(fsc[fsc.dim==2]["rank"],size = n_bars)
bar_rep = [np.random.choice(dim1_df.cell_id, size=4, replace=False) for k in np.arange(n_bars)]
# simp_weight = np.random.rand(nSimps)*(simp_dim+1)

bar_df = pd.DataFrame({"bar_id" : bar_id,
                       "bar_dim" : bar_dim,
                       "bar_birth" : bar_birth,
                       "bar_death" : bar_death,
                       "rep" : bar_rep,
                       "lifetime": bar_death-bar_birth})

bar_df["bar_dim"] = bar_df["bar_dim"].astype(str)
bar_df.head()

In [ ]:
## FOR DEV ONLY ##

## List of updates needed :
# [] read category_orders automatically
# [] legend title
# [] layer y=x behind the rest of the scatterplot
# [] show sliding window-smoothed birth/death rates
# [] show marginal distributions of birth/deaths on PDs - learned that this doesn't play well with sizing.
#    Might have to use regular subplots for this.
# [] Persistence landscapes?
# [] Color barcode bars by persistence or #simplices in minimal generator
# [] Color barcode bars by a node in the minimal generator
# [] Color barcode bars by a node in killing simplex
# [] Facet plot adjust sizing

## Updates that together make a D3 plot
# [] Barcode viz - choose color by dimension, node in minimal generator, etc.



## Persistence diagram overlayed

Create one plot showing all calculated dimensions. Best if there are three or fewer dimensions plotted. If we need to plot more than four, we should facet.

In [ ]:
## Plot overlayed persistence diagram

fig = px.scatter(bar_df, x="bar_birth", y="bar_death", color="bar_dim",
                 title="Persistence Diagrams (overlayed)", hover_data = ["rep"],
                 category_orders = {"bar_dim":[str(x) for x in np.arange(max_dim)]})

fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
    range = (0,max_filtration),
    title_text="death"
  )

fig.update_xaxes(
    range = (0,max_filtration),
    constrain='domain',
    title_text="birth",
    layer = "above traces"
)

fig.update_traces(mode='markers', marker_opacity=0.7, marker_size=12, marker_line_color = my_charcoal,
                 marker_line_width = 0.4)

fig.add_trace(
    go.Scatter(
        x=[0, max_filtration],
        y=[0, max_filtration],
        mode="lines",
        line=go.scatter.Line(color= 'rgba(187, 190, 191, .9)', width=0.5),
        showlegend=False)
)

fig.show()

# fig.write_image("fig1.pdf")


## Persistence diagram faceted


In [ ]:

fig = px.scatter(bar_df, x="bar_birth", y="bar_death", color="bar_dim", template = "plotly_white",
                 title="Persistence Diagrams (faceted)", hover_data = ["rep"],
                 color_discrete_sequence=exhact_themes.davos_colors,
                 facet_col = "bar_dim",
                 facet_col_wrap=3,
                 facet_col_spacing = 0.1,
                 facet_row_spacing = 0.3,
                 height = 300*(np.floor(max_dim/3)+1),
                 width = 800,
                 category_orders = {"bar_dim": [str(x) for x in np.arange(max_dim)]})

trace = go.Scatter(
        x=[0, max_filtration],
        y=[0, max_filtration],
        mode="lines",
        line=go.scatter.Line(color= 'rgba(187, 190, 191, .9)', width=0.5),
        showlegend=False)


# Add trace to all rows/cols, but not to empty subplots
fig.add_trace(trace, row="all", col="all", exclude_empty_subplots=True)


# Update axes
fig.update_yaxes(
    scaleratio = 1,
    range = (0,max_filtration),
    title_text="death"
  )

fig.update_xaxes(
    range = (0,max_filtration),
    constrain='domain',
    title_text="birth",
)


fig.show()

## Barcode with hover tooltip

Exactly what you would expect.

In [ ]:
fig = go.Figure()
bar_df_sorted = bar_df.sort_values(by=["bar_dim", "bar_birth"])
bar_df_sorted = bar_df_sorted.reset_index(drop=True)

for b,bar in bar_df_sorted.iterrows():
    
    if (bar_df_sorted.iloc[b-1]["bar_dim"] != bar["bar_dim"]) :
        showlegendtf = True
    else :
        showlegendtf = False
        
    fig.add_trace(go.Scatter(x=[bar["bar_birth"], bar["bar_death"]], y=[b+1, b+1],
                    mode='lines',
                    name=f'dim {bar["bar_dim"]}',
                    showlegend=showlegendtf,
                    line=dict(color=exhact_themes.davos_colors[int(bar["bar_dim"])]),
                    hovertemplate =
                        f'<i>id</i>: {bar["bar_id"]}'+
                        f'<br><b>rep</b>: {bar["rep"]}'))



# Add nice effects and such
fig.update_layout(title_text="Barcode")
fig.update_yaxes(title_text = "H_k")
fig.update_xaxes(title_text = "Filtration index")
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.update_layout(hovermode="x unified")
fig.show()

# # fig.write_image("fig1.pdf")
